In [442]:
import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio
import numpy as np

# Set the default template to dark
pio.templates.default = "plotly_dark"

In [443]:
countries_path = '../data/unprocessed/WID_countries.csv'
countries_df = pd.read_csv(filepath_or_buffer=countries_path, delimiter=';')
#countries_df['region2'].unique()
countries_df[countries_df['region2'].isna()]

,alpha2,titlename,shortname,region,region2
45,CN-RU,rural China,Rural China,NaN,NaN
46,CN-UR,urban China,Urban China,NaN,NaN
57,DE-BD,Baden,Baden,NaN,NaN
58,DE-BY,Bavaria,Bavaria,NaN,NaN
59,DE-HB,Bremen,Bremen,NaN,NaN
60,DE-HE,Hesse,Hesse,NaN,NaN
61,DE-HH,Hamburg,Hamburg,NaN,NaN
62,DE-PR,Prussia,Prussia,NaN,NaN
63,DE-SN,Saxony,Saxony,NaN,NaN
64,DE-WU,Wurttemberg,Wurttemberg,NaN,NaN


In [444]:
country_code = 'AU'
country_path = f"../data/unprocessed/WID_data_{country_code}.csv"
country = pd.read_csv(filepath_or_buffer=country_path, delimiter=';')
country.head(1)

,country,variable,percentile,year,value,age,pop
0,AU,ehfcari999,p0p100,1990,100.573195,999,i


In [445]:
meta_path = f"../data/unprocessed/WID_metadata_{country_code}.csv"
meta = pd.read_csv(meta_path, delimiter=';').drop(columns=['age', 'pop'])
meta.head(1)

,country,variable,countryname,shortname,simpledes,technicaldes,shorttype,longtype,shortpop,longpop,shortage,longage,unit,source,method,extrapolation,data_points
0,AU,accmhni992,Australia,Consumption of fixed capital attributable to m...,NaN,NaN,Average,Average income or wealth between two percentil...,individuals,The base unit is the individual (rather than t...,Adults,The population is comprised of individuals ove...,AUD,See [URL][URL_LINK]https://wid.world/document/...,WID.world estimations as a proportion of GDP b...,NaN,NaN


In [446]:
print(f"The data file contains {country.shape[1]} columns and {country.shape[0]} rows")
print(f"The metadata file contains {meta.shape[1]} columns and {meta.shape[0]} rows")
print(f"There are {country.variable.nunique()} and {meta.variable.nunique()} unique values for 'variable' in the data and metadata files respectively.")

The data file contains 7 columns and 521651 rows
The metadata file contains 17 columns and 1138 rows
There are 1138 and 1138 unique values for 'variable' in the data and metadata files respectively.


In [447]:
cols = meta.columns
nulls = [val for val in meta.isna().sum().values]
unique_vals = [meta[col].nunique() for col in meta.columns]

summary_dict = {
    'cols': cols,
    'no_of_nulls': nulls,
    'no_of_unique': unique_vals
}

summary = pd.DataFrame(summary_dict)
summary.set_index('cols', inplace=True)
summary

,no_of_nulls,no_of_unique
cols,,
country,0,1
variable,0,1138
countryname,0,1
shortname,0,286
simpledes,632,105
technicaldes,735,107
shorttype,0,14
longtype,0,14
shortpop,0,5


In [448]:
df = pd.merge(left=country, right=meta, how='left', on=['country', 'variable'], suffixes=('', '_x'))
df.nunique()

country               1
variable           1138
percentile          389
year                127
value            290527
age                  38
pop                   5
countryname           1
shortname           286
simpledes           105
technicaldes        107
shorttype            14
longtype             14
shortpop              5
longpop               5
shortage             38
longage              38
unit                  8
source               32
method               49
extrapolation         8
data_points           1
dtype: int64

In [449]:
# Create a summary of all unique variables in the dataset.
ls = []
for variable, data in df.groupby('variable'):
    summary = {
        'variable': variable,
        'shortname': data['shortname'].unique()[0],
        'pop': data['pop'].unique()[0],
        'shortpop': data['shortpop'].unique()[0],
        'shortage': data['shortage'].unique()[0],
        'unit': data['unit'].unique()[0],
        'shorttype': data['shorttype'].unique()[0],
        'longtype': data['longtype'].unique()[0]
    }
    ls.append(summary)

var_df = pd.DataFrame(ls)
var_df.head()

,variable,shortname,pop,shortpop,shortage,unit,shorttype,longtype
0,accmhni992,Consumption of fixed capital attributable to m...,i,individuals,Adults,AUD,Average,Average income or wealth between two percentil...
1,accmhni999,Consumption of fixed capital attributable to m...,i,individuals,All Ages,AUD,Average,Average income or wealth between two percentil...
2,accmhoi992,Consumption of fixed capital attributable to m...,i,individuals,Adults,AUD,Average,Average income or wealth between two percentil...
3,accmhoi999,Consumption of fixed capital attributable to m...,i,individuals,All Ages,AUD,Average,Average income or wealth between two percentil...
4,accshni992,Consumption of fixed capital attributable to o...,i,individuals,Adults,AUD,Average,Average income or wealth between two percentil...


In [450]:
# Manually list some of the variables that would be interesting to analyze.
vars_of_interest = [
    'wpwdebi999',
    'wpweali999',
    'wexpgoi999',
    'wgdproi999',
    'mgninci999',
    'agninci999',
    'agdproi999',
    'mgdproi999',
    'enfcari999',
    'enfghgi999',
    'knfcari999',
    'knfghgi999',
    'iqualii999',
    'xlcuspi999',
    'xlcusxi999',
    'npopuli999',
    'apwdebi999',
    'mpwdebi999',
    'wwealhi999',
    'wwealgi999',
    'wwealni999',
    'wwealpi999',
    'ahwealj992',
    'ahweali992',
    'bhwealj992',
    'ghwealj992',
    'ahweali999',
    'mhweali999',
    'rhwealj992',
    'shwealj992',
    'thwealj992'
]

# Extend the list of variables using some filter criteria. This is based on observations from the var_df created earlier.
var_descriptions = var_df[(var_df['variable'].isin(vars_of_interest)) | ((var_df['variable'].str.contains('gei', case=False)) & (var_df['shortage'] == 'All Ages'))]

# Create a CSV with descriptions of the variables chosen for analysis.
var_descriptions.to_csv('../data/reference/variable_descriptions.csv', index=False)

# Create a CSV of only the variable - i.e. excluding the additional variable information.
vars = pd.Series(var_descriptions['variable'].unique())
vars.to_csv('../data/reference/variables_to_analyze.csv', header=['variable'], index=False)

In [451]:
# Create a filtered dataframe including only the variables of interest.
final_df = df[df['variable'].isin(vars)].copy()

In [457]:
per_capita = var_df[(var_df['variable'].isin(vars.values)) & (var_df['unit'] == 'AUD') & (var_df['shorttype'] == 'Average')]
per_capita[per_capita['shortname'] == 'Net personal wealth']

,variable,shortname,pop,shortpop,shortage,unit,shorttype,longtype
209,ahweali992,Net personal wealth,i,individuals,Adults,AUD,Average,Average income or wealth between two percentil...
210,ahweali999,Net personal wealth,i,individuals,All Ages,AUD,Average,Average income or wealth between two percentil...
211,ahwealj992,Net personal wealth,j,equal-split adults,Adults,AUD,Average,Average income or wealth between two percentil...


In [430]:
# Create CSV files for the variables that need to be converted from local currency to USD during processing.
vars_for_currency_conversion_avg = pd.Series(final_df[(final_df['unit'] == 'AUD') & (final_df['shorttype'] == 'Average')]['variable'].unique())
vars_for_currency_conversion_avg.to_csv('../data/reference/vars_for_currency_conversion_avg.csv', header=['variable'], index=False)

vars_for_currency_conversion_total = pd.Series(final_df[(final_df['unit'] == 'AUD') & (final_df['shorttype'] == 'Total')]['variable'].unique())
vars_for_currency_conversion_total.to_csv('../data/reference/vars_for_currency_conversion_total.csv', header=['variable'], index=False)

final_df[final_df['unit'] == 'AUD']['shorttype'].unique()

array(['Average', 'Total', 'Threshold'], dtype=object)

In [431]:
gdp = df[(df['shortname'] == 'Gross domestic product') & (df['unit'] == '% of national income')][['shortname', 'year', 'value', 'shortage', 'shorttype', 'longtype', 'unit']].copy()
total_pop = df[(df['shortname'] == 'Population') & (df['shortpop'] == 'individuals') & (df['shortage'] == 'All Ages')][['year', 'value', 'shortname']]
total_pop.head(1)

,year,value,shortname
419920,1820,331000.0,Population


In [432]:
fig = go.Figure()

fig.add_trace(
        go.Scatter(
        x=gdp['year'],
        y=gdp['value']
        )
    )

fig.update_layout(
    title='Gross Domestic Product over time'
)

fig.show()

In [433]:
shortname_list = [
    'Defense',
    'Economic affairs',
    'Education',
    'Environmental protection',
    'Health',
    'Housing and community amenities',
    'Public order and safety',
    'Recreation and culture',
    'Social protection'
]

cols_to_keep = ['year', 'value', 'shortname', 'shorttype', 'longtype', 'variable']

a = df[(df['shortname'].isin(shortname_list)) & (df['shorttype'] == 'Total')][cols_to_keep]

total_pop = df[(df['shortname'] == 'Population') & (df['shortpop'] == 'individuals') & (df['shortage'] == 'All Ages')][['year', 'value']]
fx = df[df['shortname'] == 'PPP conversion factor, LCU per USD'][['year', 'value']].rename(columns={'value': 'local_currency_per_usd'})
public_spending = df.loc[(df['shortname'] == 'Total Public Spending (excluding interest payment)') & (df['shorttype'] == 'Total')][['year', 'value']].rename(columns={'value': 'total_public_spending'})

In [434]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=public_spending['year'],
    y=public_spending['total_public_spending']
))

fig.update_layout(
    title='Total public spending'
)

fig.show()

In [435]:
fig = go.Figure()

for _, group in a.groupby('shortname'):

    if len(group['variable'].unique()) > 1:
        group = group[group['variable'] == group['variable'].unique()[0]]

    name = group['shortname'].unique()[0]
    fig.add_trace(
        go.Scatter(
            x=group['year'],
            y=group['value'],
            name=name,
            mode='lines',
            stackgroup='one',
            groupnorm='percent'
        )
    )

fig.update_layout(
    title='Share of public spending'
)

fig.show()

In [436]:
fig = go.Figure()

for _, group in a.groupby('shortname'):

    if len(group['variable'].unique()) > 1:
        group = group[group['variable'] == group['variable'].unique()[0]]
        
    data = group.merge(total_pop, how='inner', on='year', suffixes=('', '_pop')).merge(fx, how='inner', on='year')
    data['per_capita_local'] = data['value'] / data['value_pop']
    data['per_capita_usd'] = data['per_capita_local'] / data['local_currency_per_usd']

    name = data['shortname'].unique()[0]
    fig.add_trace(
        go.Scatter(
            x=data['year'],
            y=data['per_capita_local'],
            name=name,
            mode='lines+markers'
        )
    )

fig.update_layout(
    title='Per capita public spending'
)

fig.show()

In [437]:
variables = [
    'wpwdebi999',
    'wpweali999'
]

private_debt_wealth = df.loc[df['variable'].isin(variables)].copy()

fig = go.Figure()

for _, group in private_debt_wealth.groupby('variable'):
    fig.add_trace(
        go.Scatter(
            x=group['year'],
            y=group['value'],
            name=group['shortname'].unique()[0]
        )
    )

fig.update_layout(
    title='Private debt and private wealth as a % of national income'
)

fig.show()

In [438]:
wealth = [
    'Net personal wealth'
]

ages = [
    'All Ages',
    'Adults'
]

df[(df['shortname'].isin(wealth)) & (df['shortage'] == 'All Ages')]['shorttype'].unique()
df[(df['shortname'].isin(wealth)) & (df['shortage'].isin(ages))]['variable'].unique()

array(['ahwealj992', 'ahweali992', 'bhwealj992', 'ghwealj992',
       'ahweali999', 'mhweali999', 'rhwealj992', 'shwealj992',
       'thwealj992'], dtype=object)

In [439]:
# Can Average and Total subsets be easily combined into one variable? Does it makes sense?

df_list = []
my_set = set()

for name, data in df.groupby('shortname'):
    if np.all(np.isin(['Average', 'Total'], data['shorttype'].unique())):
        my_set.add(name)
    
    shorttype_avg = data[data['shorttype'] == 'Average'][['shortname', 'year', 'value']]
    shorttype_total = data[data['shorttype'] == 'Total']
    combined = pd.merge(left=shorttype_total, right=shorttype_avg, on=['shortname', 'year'], how='left')
    df_list.append(combined)

for item in my_set:
    print(item)
#df_list[0]

Environment protection
Gross domestic product
Private financial assets excluding currency & deposits
Gross operating surplus and miscellaneous
Net secondary income/Net saving of financial corporations
Gross foreign assets
National income
Government agricultural land
Other primary income paid to the rest of the world
Net mixed income of households and NPISH
Private natural capital
Government dwellings
Corporate non-financial assets 
Government financial assets excluding cash
Recreation, culture and religion
Public order and safety
Taxes on products and production
Gross primary income of financial corporations
Gross national savings
Net operating surplus of corporations
Private equity, fund shares & offshore wealth
Exports of goods and services
Remittances paid to the rest of the world
Portfolio equity liabilities
Property income paid from the rest of the world
Government housing assets
Total Public Spending (excluding interest payment)
Capital transfers paid to  the rest of the world
Pe

In [440]:
my_list = []
for var, data in df.groupby('variable'):
    if var in vars.values:
        my_list.append(
            {
                'variable': var,
                'shortname': data['shortname'].unique()[0],
                'shorttype': data['shorttype'].unique()[0],
                'unit': data['unit'].unique()[0]
            }
        )

pd.set_option('display.max_rows', None)
pd.DataFrame(my_list).sort_values('shortname')

,variable,shortname,shorttype,unit
0,adefgei999,Defense,Average,AUD
53,wdefgei999,Defense,Wealth-income ratio,% of national income
29,mdefgei999,Defense,Total,AUD
1,aecogei999,Economic affairs,Average,AUD
54,wecogei999,Economic affairs,Wealth-income ratio,% of national income
30,mecogei999,Economic affairs,Total,AUD
58,wedugei999,Education,Wealth-income ratio,% of national income
5,aedugei999,Education,Average,AUD
34,medugei999,Education,Total,AUD
31,medpgei999,Education: Primary,Total,AUD
